In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
import os
import h5py
import numpy as np
import pandas as pd
from data_helpers import grouper
from TempConv import determine_fit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX TITAN X (CNMeM is enabled with initial size: 95.0% of memory, cuDNN not available)


In [9]:
nn_params = {
    'bs' : 256,
    'eps' : 15,
    'lr' : 0.0005,
    'kernel' : 2,
    'nb_filter' : 5,
    'window' : 60,
    'offset' : 10,
    'nb_test' : 3,
    'nb_trains' : 2,
    'verbose' : False
}

In [4]:
## get and format data
lfp_file = h5py.File('lfp_power.hdf5', 'r')
neural_data = np.asarray(lfp_file['lfp_power']) # iterate through powerbands
tetrodes = grouper(neural_data, 24)

head_signals_h5 = h5py.File('all_head_data.hdf5', 'r')
idx_start, idx_stop = [8,9]
head_signals = np.asarray([np.asarray(head_signals_h5[key]) for key in head_signals_h5.keys()][0:9]).T[:,idx_start:idx_stop]
print('head_signals shape: ', head_signals.shape)

head_signals_keys = list(head_signals_h5.keys())[0:9][idx_start:idx_stop]
head_signals_int = ['yaw_abs', 'roll_abs', 'pitch_abs']
print('head_signals_keys intuitive: ', head_signals_int)

head_signals shape:  (80223, 1)
head_signals_keys intuitive:  ['yaw_abs', 'roll_abs', 'pitch_abs']


In [10]:
stats = {}

In [12]:
# iterate Xs
for tetrode_idx in range(tetrodes.shape[0]):
    tetrode = tetrodes[tetrode_idx].T
    
    if tetrode_idx >= 1: break
    
    # iterate ys
    for head_signal_idx in range(head_signals.shape[1]):
        R2r_arr = {
            'R2s' : [],
            'rs' : []
        }
        
        for i in range(nn_params['nb_trains']): # replace with k-fold? n k-folds?
            head_signal = head_signals[:,head_signal_idx]
            R2, r = determine_fit(tetrode, head_signal, [head_signals_int[head_signal_idx]], nn_params)
            
            R2r_arr['R2s'].append(R2[0])
            R2r_arr['rs'].append(r[0])
        
        stats['tetrode_{}_head_signal_{}'.format(tetrode_idx, head_signal_idx)] = R2r_arr

Train on 39757 samples, validate on 39756 samples
Epoch 1/15
39757/39757 [==============================] - 3s - loss: 61.0614 - mean_squared_error: 5877.2855 - val_loss: 62.0889 - val_mean_squared_error: 5776.8098

In [ ]:
stats

In [ ]:
print(0)